# Fundamentals of Software Systems - SE Part I Assignment

By Andy Wiemeyer and Lucius Bachmann

### Setup tools
* checkout repo
* initialize Git utility
* You can recreate the Repository object with other parameters to analyze different time periods.
  The last year was used that the setup is fast.

In [ ]:
import os.path
from datetime import datetime
from os import path, mkdir

import pandas as pd
import plotly.express as px
from pydriller import Repository, Git

repo_remote_path = 'https://github.com/mastodon/mastodon.git'
repo_path = 'mastodon'
repo_checkout_path = f'{repo_path}/{repo_path}'
filepath = 'app'

if not path.exists(repo_path):
    mkdir(repo_path)

repo = Repository(repo_remote_path, clone_repo_to=repo_path, since=datetime.fromisoformat('2022-10-01'), filepath=filepath)
# clone repo if necessary
for commit in repo.traverse_commits():
    break
git = Git(repo_checkout_path)

### Checkout repo at tag v3.5.3

In [ ]:
tag = git.get_commit_from_tag('v3.5.3')
git.checkout(tag.hash)

## 1 Complexity Hotspots

1. You must consider only the app folder from the Mastodon repository
(i.e., https://github.com/mastodon/mastodon).

-> nothing to do

2. Decide on the granularity of your analysis of software entities (e.g., source code
files); describe why you selected this specific granularity.

For this analysis the granularity of source code files is used. It is an easy unit to perform measurements on. The mastodon repository contains a ruby on rails application with some javascript for the frontend. In both languages it's possible to define multiple classes in one file. Without performing a programming language specific analysis, it's not possible to measure smaller units.


3. Create a list of all these entities, as they appear in the latest stable release of
Mastodon (i.e., tag v3.5.3).

In [ ]:
full_file_paths_all_dirs = git.files() #full path of all files in the analyzed commit
subdirectory_start = "/" + repo_checkout_path + "/"
full_file_paths = [path for path in full_file_paths_all_dirs if subdirectory_start+"app/" in path]
subdirectory_start_index = full_file_paths[0].find(subdirectory_start) + len(subdirectory_start)
subdirectory_prefix = full_file_paths[0][:subdirectory_start_index]#used later
file_paths = [path[subdirectory_start_index:] for path in full_file_paths] #paths relative to analyzed subdirectory

In [ ]:
file_names = [path[max(0,path.rfind("/"))+1:] for path in full_file_paths]
print(f"Amount of different files with equal name: {len(file_names)-len(set(file_names))}")
counted_file_names = {}
for name in file_names:
    if name not in counted_file_names:
        counted_file_names[name] = 1
    else:
        counted_file_names[name] += 1
print([f"{name}: {count}" for name, count in counted_file_names.items() if count>1])

A lot of different files have the same name (see above) so we use the paths to identify files:

In [ ]:
print(file_paths)

List the file endings of the paths used in this analysis

In [ ]:
from pandas import DataFrame
from collections import Counter

extensions = Counter([path.splitext(entity)[1] for entity in file_paths])

colname_nr_of_occurences = 'nr of occurrences'
df_extensions = DataFrame(extensions.items(), columns=['file ending', colname_nr_of_occurences])
display(df_extensions.sort_values(by=colname_nr_of_occurences, ascending=False))

4. Decide on the type of complexity you want to measure for your software entities
and explain why you selected this type.

To decide which metric would be a good indicator for complexity, a file was chosen to show the metrics the lizard library provides.

In [ ]:
import lizard

filename = 'mastodon/mastodon/app/workers/scheduler/indexing_scheduler.rb'
file = open(filename, mode='r')
analysis = lizard.analyze_file.analyze_source_code(filename, file.read())
print(f'of file {filename}')
print(f'nr of functions: {len(analysis.function_list)}')
print(f'cyclomatic complexity: {analysis.CCN}')
print(f'lines of code: {analysis.nloc}')
print(f'token_count: {analysis.token_count}')
try:
    print(f'deepest nesting level: {analysis.ND}')
except AttributeError:
    print(f'deepest nesting level threw an error')



In the lecture we learned that the number of lines of code is a proven measurement for complexity.
Pydriller already measures this, thus the number of lines of code is used as measurement for complexity.

5. Decide on a timeframe on which you want to base your analysis and explain the
rationale of your choice.

In a first step, the activity on the repository is analysed.

In [ ]:
repo = Repository(repo_remote_path, clone_repo_to=repo_path)
commit_dates = []

for commit in repo.traverse_commits():
    commit_dates.append(commit.committer_date.strftime('%Y-%m-%d'))

df_commit_dates = DataFrame(Counter(commit_dates).items(), columns=['date', 'nr_of_commits'])
line = px.line(df_commit_dates, x='date', y='nr_of_commits')
line.update_layout(font_size=18)
line.show()

Here we see that the development activity had a peak in 2017, and would be a little higher in the year 2020.
Because we saw that with a timeframe between 2019 and 2022 we see a significant number of defects,
we chose the timeframe of the last 3 years.

If a complexity hotspot was found which was created a long time ago and
was not changed for a long time, then the analysis would not add value.
This analysis focuses on the last year.

In [ ]:
since = datetime.fromisoformat('2019-11-08')
to = datetime.fromisoformat('2022-11-08')

repo = Repository(repo_remote_path, clone_repo_to=repo_path, since=since, to=to, filepath=filepath)
git = Git(repo_checkout_path)

6. For each entity in the system, measure its complexity and the number of changes
(in the given timeframe). Merge these two pieces of information together to cre-
ate a candidate list of problematic hotspots in the app part of Mastodon.

In [ ]:
path_to_results = './analysis_data.csv'
if not os.path.isfile(path_to_results): #check if we have already saved the results
    analysis_df = pd.DataFrame( #create dataframe with
        {'Name':file_names, 'Full Path':full_file_paths}, #data columns
        index=file_paths) #and index

    #Compute complexities
    complexity_comp_exceptions = {} #to analyse what caused error (mostly images)
    for idx in analysis_df.index:
        try:
            with open(analysis_df.loc[idx, 'Full Path'], 'r') as file:
                analysis_df.loc[idx, 'Complexity'] = len(file.readlines())
        except Exception as e:
            complexity_comp_exceptions[idx]=e
    analysis_df.to_csv(path_to_results)

    # Count amount of times changed
    analysis_df['CommitPath'] = analysis_df.index #For tracking the path that a file has in the current commit if it was moved
    analysis_df['Amount of changes'] = 0 #set change counter to 0
    for commit in reversed(list(repo.traverse_commits())):
        for file in commit.modified_files:
            idx = analysis_df[analysis_df['CommitPath']==file.new_path].index
            analysis_df.loc[idx, 'Amount of changes'] += 1
            analysis_df.loc[idx, 'CommitPath'] = file.old_path #update commit path
    analysis_df['Initial Path'] = analysis_df['CommitPath']
    analysis_df.drop(columns='CommitPath', inplace=True) #get rid of temporary indexing
    analysis_df.to_csv(path_to_results)

In [ ]:
analysis_df = pd.read_csv(path_to_results, index_col=[0])
# show results
#analysis_df['Complexity'].sort_values(ascending=False)[:10] #Print top 10 entries
#analysis_df['Amount of changes'].sort_values(ascending=False)[:10] #Print top 10 entries

7. Visualize the hotspots with a visualization of your choice.

In [ ]:
complexity_hist = px.histogram(analysis_df, x='Complexity', hover_data={"path": analysis_df.index})
complexity_hist.show() #exponential dist in complexity -> use log axis
change_amount_hist = px.histogram(analysis_df, x='Amount of changes', hover_data={"path": analysis_df.index})
change_amount_hist.show() #exponential dist in amount of changes -> use log axis

comparison_scatter = px.scatter(analysis_df, x='Complexity', y='Amount of changes',
                 hover_data={"path": analysis_df.index})
comparison_scatter.show()

comparison_scatter_log = px.scatter(analysis_df, x='Complexity', y='Amount of changes',
                 log_x=True, log_y=True,
                 hover_data={"path": analysis_df.index})
comparison_scatter_log.show()

In [ ]:
display(analysis_df.sort_values(by=['Amount of changes','Complexity'], ascending=False)[:20])

8. Analyze six candidate hotspots (not necessarily the top ones) through:

In [ ]:
"""
computes the complextity trends for a list of file paths
return: dictionary with paths as key and a list containing a complexity measurements from oldest to newest commit as value
"""
def compute_complexity_trends(file_paths):
    complexity_trends = {path: [] for path in file_paths}
    commit_paths = {path:path for path in file_paths} #For tracking the path that a file has in the current commit if it was moved

    for commit in reversed(list(repo.traverse_commits())):#TODO: use whole list
        git.checkout(commit.hash)

        #add trend values for current commit
        for key_path, value_path in commit_paths.items():
            try:
                full_path = subdirectory_prefix+value_path
                with open(full_path, 'r') as file:
                    complexity_trends[key_path].insert(0,len(file.readlines()))
            except TypeError as e:
                if e.args[0] == 'can only concatenate str (not "NoneType") to str':
                    complexity_trends[key_path].insert(0,0)
                else:
                    raise Exception
            except Exception as e:
                print("Issue with complexity trend computation: ", e)

        #update commit paths
        for file in commit.modified_files:
            for key_path, value_path in commit_paths.items():
                if value_path==file.new_path and file.new_path!=None:
                    commit_paths[key_path]=file.old_path

    git.checkout(tag.hash) #return to initial checkout
    return complexity_trends


In [ ]:
hotspot_candidates = [
    'app/javascript/fonts/roboto/roboto-medium-webfont.svg', #Complexity outlier, never changes
    'app/models/status.rb', #Very high change rate somewhat high complexity
    'app/javascript/styles/mastodon/components.scss', #Very high complexity, very high amount of changes
    'app/javascript/mastodon/locales/ca.json',
    'app/helpers/application_helper.rb',
    'app/services/activitypub/process_status_update_service.rb']
complexity_trends = compute_complexity_trends(hotspot_candidates)

In [ ]:
def plot_complexity_trend(index):
    label, complexity_list = list(complexity_trends.items())[index]
    commits_num = list(range(len(complexity_list)))
    complexity_trend_line = px.line(x=commits_num, y=complexity_list, title=f"Complexity trend for {label}",
                                    labels={'x': 'number of commits', 'y': 'complexity [LOC]'})
    complexity_trend_line.show()
    print(f"Final complexity: {int(analysis_df['Complexity'][label])} LOC" )
    print(f"Amount of changes: {analysis_df['Amount of changes'][label]}" )

In [ ]:
def get_git_log(filepath: str) -> DataFrame:
    global commit
    repo_file = Repository(path_to_repo=repo_checkout_path, since=since, to=to,
                           filepath=filepath)
    git_log_file = []
    for commit in repo_file.traverse_commits():
        insertions = 0
        deletions = 0
        for file in commit.modified_files:
            if file.new_path == filepath:
                insertions = file.added_lines
                deletions = file.deleted_lines
        git_log_file.append((commit.hash[:7], commit.msg, insertions, deletions))
    return DataFrame(git_log_file, columns=['hash', 'message', 'insertions', 'deletions'])

In [ ]:
from glob import glob

def show_stats_for_glob(glob_str: str):
    def create_file_row(file_path: str):
        analysis = lizard.analyze_file(file_path)
        return file_path.replace(repo_checkout_path, ''), analysis.nloc
    files = [create_file_row(filepath) for filepath in glob(f'{repo_checkout_path}/{glob_str}', recursive=True)]
    display(DataFrame(files, columns=['filename', 'nloc']).sort_values(by=['nloc'], ascending=False))

Candidate 1: app/javascript/fonts/roboto/roboto-medium-webfont.svg

The 'roboto-medium-webfont.svg' file is an extreme outlier. It has very high complexity and never changes.
From the file name we can easily see that it is not a troublesome hotspot.
The file needs to have a lot of lines of code because it defines the shape of characters from a font.
None of these lines ever need to change though. It is not a hotspot and we do not need to take any action.

In [ ]:
plot_complexity_trend(0)


Candidate 2: app/models/status.rb

From the trend we see that the complexity increased gradually, with some decreases inbetween.
Presumably these are refactorings, indicating that the file does indeed require some maintenance work.

In [ ]:
plot_complexity_trend(1)

Candidate 3: app/javascript/styles/mastodon/components.scss

In [ ]:
plot_complexity_trend(2)

In [ ]:
git_log = get_git_log('app/javascript/styles/mastodon/components.scss')
display(git_log[-20:])

It seems that the file is changed for unrelated features. This assumes that for a lot of css in this file
for different responsibilities.
This raises the question if the scss files are separated at all.
For that the nloc of the scss files needs to be analysed.

In [ ]:
show_stats_for_glob('**/*.scss')


Now one can see that the scss is separated in some files, but the components.css is still
a lot larger than all the other scss files.
The commits 2de44d3 and 2de5128 show that 2 pull requests were needed to fix the same regression.
The function `application_helper.rb::react_component` indicates that there are react components
in this repository, which would allow to split and scope the scss. This was not done and thus
the components.scss file is large and handles the styling concerns of more than one component.
This already lead to regressions. Components.scss is a complexity hotspot.


Candidate 4: app/javascript/mastodon/locales/ca.json


In [ ]:
plot_complexity_trend(3)

ca.json and the other locales files contain some translations for the languages mastodon supports.
So these files contain a lot of lines and change on 3 occasions:

1. If a new translation key is added
2. If the translation for a translation key is changed
3. If the key for a translation needs to be changed


In [ ]:
git_log = get_git_log('app/javascript/mastodon/locales/ca.json')
display(git_log[:30])

From the commit messages we can see that mastodon uses Crowdin to manage their translations.
It's good that the translations can be changed independently. On the other hand we see in commits
1392741 and db04dfc we see that sometimes the authors forget to add the translation keys to the
locales files. This is not really a hotspot, but shows that high number of lines of code and a
high change rate do not always indicate a hotspot.

Candidate 5: app/helpers/application_helper.rb


In [ ]:
plot_complexity_trend(4)

From the name one does not really know what this module should do.
The amount of changes is not large, but the file is in the upper right corner of the scatter plot above.
Because the name does not really describe the responsibility of the module, an analysis of the method names was made.


In [ ]:
filename = 'mastodon/mastodon/app/helpers/application_helper.rb'
file = open(filename, mode='r')
analysis = lizard.analyze_file.analyze_source_code(filename, file.read())
functions = [function_info.name for function_info in analysis.function_list]
display(functions)

Some functions seem to be helpers for rendering the ui, like `active_nav_class`, `active_link_to`, `friendly_number_to_human`, `favicon_path`
, `react_component`.
Others seem to contain business logic like `grouped_scopes`, `open_registrations?`.
Its not the most important hotspot, but it would be good to split this module into separate modules.

Candidate 6: app/services/activitypub/process_status_update_service.rb

In [ ]:
plot_complexity_trend(5)

This file started with a high complexity, and its complexity stayed at the same level.
The changes made to the file did not change the complexity drastically.

In [ ]:
display(get_git_log('app/services/activitypub/process_status_update_service.rb'))

The git log shows a lot of fixes in this file. This makes it a complexity hotspot.


## 2 Temporal/Logical Coupling

1. Determine what could be cases of temporal/logical coupling and generate a list
of candidates with a set of coupled entities.

In this analysis we consider logical grouping by commits.
According to the [guidelines for pull requrests](https://github.com/mastodon/mastodon/blob/97f657f8181dc24f6c30b6e9f0ce52df827ac90f/CONTRIBUTING.md#pull-requests),
small pull requests should be preferred, thus it is assumed that they contain only the necessary changes for one "Add", "Change", "Deprecate", "Remove" or "Fix",
and can be viewed as logically grouped. To merge the pull request, a squash commit is added to the main branch. So for this analysis we can use the commits
on the main branch to detect logical coupling between files.

Here the sets of logically coupled is created.

In [ ]:
from pandas import DataFrame
from typing import List, Dict
from dataclasses import dataclass


@dataclass
class ChangeSet:
    files: Dict[str,int]
    counter: int

    def __init__(self, files: List[str]):
        self.files={f:1 for f in files}
        self.counter = 0

    def __hash__(self) -> int:
        return hash(", ".join(self.get_files()))


    def __eq__(self, o: object) -> bool:
        if not isinstance(o, ChangeSet):
            return False
        # noinspection PyUnresolvedReferences
        return self.get_files() == o.get_files()

    def to_tuple(self, total_commits_file1 : int, total_commits_file2 : int):
        real_number_of_coupled_commits = self.counter / 2
        file1_coupling = real_number_of_coupled_commits / total_commits_file1
        file2_coupling = real_number_of_coupled_commits / total_commits_file2
        max_coupling = max(file1_coupling, file2_coupling)
        return self.get_files()[0], total_commits_file1, file1_coupling, self.get_files()[1], total_commits_file2, file2_coupling, real_number_of_coupled_commits, max_coupling

    def get_files(self):
        return sorted(self.files.keys())

    def changeset_changed(self):
        self.counter += 1
changed_together_filename = './changed_together.csv'

col_name_nr_changed_together = 'nr_changed_together'
col_name_coupling_file1 = 'file1_coupling'
col_name_coupling_file2 = 'file2_coupling'
col_name_max_coupling = 'max_coupling'
changed_together_columns = ['file1', 'file1_total_commits', col_name_coupling_file1 , 'file2', 'file2_total_commits', col_name_coupling_file2, col_name_nr_changed_together, col_name_max_coupling]
dataframe_changed_together = DataFrame([], columns=changed_together_columns)

if not path.exists(changed_together_filename):
    files_changed_together : Dict[ChangeSet,ChangeSet] = {}
    commits_on_file_counter: Dict[str,int] = {}
    for commit in repo.traverse_commits():
        for file in commit.modified_files:

            if not file.new_path in commits_on_file_counter:
                commits_on_file_counter[file.new_path] = 0
            commits_on_file_counter[file.new_path] += 1

            for changed_together in commit.modified_files:
                if file.new_path is None or changed_together.new_path is None:
                    continue
                if not file.new_path.__contains__('app/') or not changed_together.new_path.__contains__('app'):
                    continue
                if not file.new_path == changed_together.new_path:
                    change_set = ChangeSet([file.new_path, changed_together.new_path])
                    if change_set in files_changed_together:
                        change_set = files_changed_together[change_set]
                    else:
                        files_changed_together[change_set] = change_set
                    change_set.changeset_changed()

    changed_together_tuples = []
    for change_set in files_changed_together.values():
        total_commits_file1 = commits_on_file_counter[change_set.get_files()[0]]
        total_commits_file2 = commits_on_file_counter[change_set.get_files()[1]]
        changed_together_tuples.append(change_set.to_tuple(total_commits_file1, total_commits_file2))

    dataframe_changed_together = DataFrame(changed_together_tuples, columns=changed_together_columns)
    dataframe_changed_together = dataframe_changed_together.sort_values(by=[col_name_nr_changed_together], ascending=False)
    dataframe_changed_together.to_csv(changed_together_filename)

dataframe_changed_together = pd.read_csv(changed_together_filename, index_col=[0,1])



print(f'most common changed together:')
display(dataframe_changed_together[:10])

As expected, the locales change often together. Thus, we need to remove the locales from the analysis.
Then the data is filtered by files which are at least 5 times changed together, and sorted by the max_coupling value.

In [ ]:
from tabulate import tabulate
allowed_file_endings = ['.rb', '.js', '.haml', '.erb']

def contains_allowed_file_type(change_set: List) -> bool:
    for file in change_set:
        ending = path.splitext(file)[1]
        if allowed_file_endings.__contains__(ending):
            return True
    return False

changed_together_filtered = [(row[1], row[2], row[3], row[4], row[5], row[6], row[7], row[8]) for row in dataframe_changed_together.to_records(index=True) if contains_allowed_file_type([row[1], row[4]])]
dataframe_changed_together_filtered = DataFrame(changed_together_filtered, columns=changed_together_columns)
dataframe_changed_together_filtered = dataframe_changed_together_filtered[dataframe_changed_together_filtered['nr_changed_together'] > 5].sort_values(by=[col_name_max_coupling], ascending=False)

tabulate(dataframe_changed_together_filtered, headers=changed_together_columns, tablefmt="html")

2. Visualize these candidate sets of couple entities with a visualization of your
choice.

In [ ]:
complexity_hist = px.histogram(dataframe_changed_together_filtered, x=col_name_nr_changed_together, log_y=True)
complexity_hist.update_layout(font_size=18)
complexity_hist.show()

The histogram visualises that there is only one file pair of the analyzed file types (which excludes the locales), which was changed a lot together.
The others were only changed together 6 times in the last 3 years.

In [ ]:
list_for_bar_plot = [(f'{row[1]} +  {row[4]}', row[8]) for row in dataframe_changed_together_filtered.to_records(index=True)]
dataframe_for_bar_plot = DataFrame(list_for_bar_plot, columns=['files', col_name_max_coupling])

bar = px.bar(dataframe_for_bar_plot[:20], x=col_name_max_coupling, y='files')
bar.update_layout(font_size=18)
bar.show()

In this horizontal bar charts, the pairs are sorted by the maximal coupling value of the two files involved.
There are the following groups of pairs:
* Coupled frontend (javascript or haml files)
* Coupled backend controllers
* Coupled frontend state management files (for redux which we can see from the actions/compose.js and the actions/reducers.js files)
* Coupled backend helpers with frontend they render

3. For three set candidates in the list:
• analyze and explain why these entities are coupled;
• describe how important it would be to fix them, and any ideas for their
improvement.


Canditate set 1 (app/controllers/follower_accounts_controller.rb, app/controllers/following_accounts_controller.rb):

The values retrieved for the set:

In [ ]:
tabulate(dataframe_changed_together_filtered[dataframe_changed_together_filtered['file1'] == 'app/controllers/follower_accounts_controller.rb'], headers=changed_together_columns, tablefmt="html")

The joined git history:

In [ ]:

def join_git_logs(git_log_1: DataFrame, git_log_2: DataFrame) -> DataFrame:
    return git_log_1.set_index('hash').join(git_log_2.set_index('hash'), how='outer', rsuffix='_right')

git_log_1 = get_git_log('app/controllers/follower_accounts_controller.rb')
git_log_2 = get_git_log('app/controllers/following_accounts_controller.rb')
display(join_git_logs(git_log_1, git_log_2))

This git log suggests that the changing parts of the two files are copy pasted.

In [ ]:
print(os.system('git diff --no-index mastodon/mastodon/app/controllers/follower_accounts_controller.rb mastodon/mastodon/app/controllers/following_accounts_controller.rb'))

The 2 files are coupled because they need to fulfil the same responsibility in a different context.
The coupling could be avoided if the common logic is extracted into a third class.

Canditate set 2 (app/javascript/mastodon/actions/compose.js, app/javascript/mastodon/reducers/compose.js):

In [ ]:
tabulate(dataframe_changed_together_filtered[dataframe_changed_together_filtered['file1'] == 'app/javascript/mastodon/actions/compose.js'], headers=changed_together_columns, tablefmt="html")

In [ ]:
git_log_1 = get_git_log('app/javascript/mastodon/actions/compose.js')
git_log_2 = get_git_log('app/javascript/mastodon/reducers/compose.js')
display(join_git_logs(git_log_1, git_log_2))

The actions and the reducers are tightly coupled in the redux state management, because the reducers need to calculate the new
state given the old state and the action.
Another state management system might allow to put related concerns into the same file.

Candidate set 3 (app/helpers/statuses_helper.rb, app/views/statuses/_simple_status.html.haml):

In [ ]:
tabulate(dataframe_changed_together_filtered[dataframe_changed_together_filtered['file1'] == 'app/helpers/statuses_helper.rb'], headers=changed_together_columns, tablefmt="html")

In [ ]:
git_log_1 = get_git_log('app/helpers/statuses_helper.rb')
git_log_2 = get_git_log('app/views/statuses/_simple_status.html.haml')
display(join_git_logs(git_log_1, git_log_2))

Here the statuses_helper provides logic for the haml template.
Because the status_helper is only shared among 2 related templates, this is not an issue.

1. Decide on how you want to detect entities that had defects in the past (e.g.,
commit message analysis vs. issue tracking system analysis) and motivate your
choice.

We decided to look at commit messages. Whenever a file was changed in a commit with message that contains a string like "bug" or "fix" (see "indicator_list" below), we increment a counter for that file.

2. Determine defective hotspots among the entities in the timeframe that you pre-
viously selected (i.e., consider only defects in the selected timeframe). What
conclusions can you draw from this?


In [ ]:
indictor_list = ["fix", "bug"]
if not 'Defects' in analysis_df: #check if we have already saved the results

    #Compute defect count
    analysis_df['CommitPath'] = analysis_df.index #For tracking the path that a file has in the current commit if it was moved
    analysis_df['Defects'] = 0 #set change counter to 0

    for commit in reversed(list(repo.traverse_commits())):
        fixes_bug = any([indicator in commit.msg.lower() for indicator in indictor_list])
        for file in commit.modified_files:
            idx = analysis_df[analysis_df['CommitPath']==file.new_path].index
            analysis_df.loc[idx, 'CommitPath'] = file.old_path #update commit path
            if fixes_bug:
                analysis_df.loc[idx, 'Defects'] += 1

    analysis_df.drop(columns='CommitPath', inplace=True) #get rid of temporary indexing
    analysis_df.to_csv(path_to_results)

analysis_df['Defects'].sort_values(ascending=False)[:10] #Print top 10 entries

3. Determine complexity hotspots at the beginning of your timeframe, then corre-
late them with the defects they have presented throughout the entire timeframe.
Is there a correlation? Why do you think this is the case?

In [ ]:
#Find complexity hotspots at the start
initial_commit = list(repo.traverse_commits())[0]
git.checkout(initial_commit.hash)# go to first commit

complexity_comp_exceptions = {} #to analyse what caused error (mostly images)
for idx in analysis_df.index:
    try:
        if type(analysis_df.loc[idx, 'Initial Path']) != str: #if the file is deleted, initial path is a float, nan
            analysis_df.loc[idx, 'Initial Complexity'] = 0
        else:
            full_path = subdirectory_prefix+analysis_df.loc[idx, 'Initial Path']
            with open(full_path, 'r') as file:
                analysis_df.loc[idx, 'Initial Complexity'] = len(file.readlines())
    except Exception as e:
        complexity_comp_exceptions[idx]=e
#print("Exceptions: \n", complexity_comp_exceptions)
analysis_df.to_csv(path_to_results)

git.checkout(tag.hash)# revert to latest commit
display(analysis_df.sort_values(by=['Initial Complexity', 'Defects'], ascending=False)[:20])

In [ ]:
comparison_scatter = px.scatter(analysis_df, x='Initial Complexity', y='Defects',
                 hover_data={"path": analysis_df.index})
comparison_scatter.show()

comparison_scatter_zoomed = px.scatter(analysis_df, x='Initial Complexity', y='Defects',
                 hover_data={"path": analysis_df.index})
comparison_scatter_zoomed.update_yaxes(range = [0,30])
comparison_scatter_zoomed.update_xaxes(range = [0,1100])
comparison_scatter_zoomed.show()

reduced_df = analysis_df.dropna(subset=['Initial Path'])
reduced_df = reduced_df[~((reduced_df['Initial Path'].str.endswith('.scss'))|(reduced_df['Initial Path'].str.endswith('.json')))]
comparison_scatter_zoomed_filtered = px.scatter(reduced_df, x='Initial Complexity', y='Defects',
                 hover_data={"path": reduced_df.index})
comparison_scatter_zoomed_filtered.update_yaxes(range = [0,30])
comparison_scatter_zoomed_filtered.update_xaxes(range = [0,1100])
comparison_scatter_zoomed_filtered.show()


We see that there is a slight correlation.

4. What conclusions can you draw from the relationship between defective hotspots
and complexity hotspots in Mastodon? And on these two metrics in general?
